In [2]:
pip install requests backoff

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached certifi-2022.9.24-py3-none-any.whl (161 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
  Using cached urllib3-1.26.12-py2.py3-none-any.whl (140 kB)
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/ho

In [3]:
import requests
import json
import backoff
import random
import logging

In [7]:
log = logging.getLogger()
log.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
ch = logging.StreamHandler()
ch.setFormatter(formatter)
log.addHandler(ch)

In [4]:
# https://docs.awesomeapi.com.br/
url = "https://economia.awesomeapi.com.br/json/last/USD-BRL"
ret = requests.get(url)

In [61]:
# if ret -> identifies automatically if the return is good.
if ret:
    print(ret)
else:
    print('request error')

<Response [200]>


In [62]:
json.loads(ret.text)

{'USDBRL': {'code': 'USD',
  'codein': 'BRL',
  'name': 'Dólar Americano/Real Brasileiro',
  'high': '5.2712',
  'low': '5.2705',
  'varBid': '0.0008',
  'pctChange': '0.02',
  'bid': '5.2703',
  'ask': '5.2713',
  'timestamp': '1666231086',
  'create_date': '2022-10-19 22:58:06'}}

In [63]:
usd = json.loads(ret.text)['USDBRL']
print(f"20 dollars costs today: {float(usd['bid']) * 20} reais")

20 dólars costs today: 105.40599999999999 reais


In [50]:
def quotation(value, currency):
    url = f"https://economia.awesomeapi.com.br/json/last/{currency}"
    ret = requests.get(url)
    currency_price = json.loads(ret.text)[currency.replace('-','')]
    print(f"{value} {currency:3} costs today: {float(currency_price['bid']) * 20} {currency[-3:]}")

In [51]:
quotation(20, 'USD-BRL')

20 USD-BRL costs today: 105.40599999999999 BRL


In [52]:
quotation(20, 'JPY-BRL')

20 JPY-BRL costs today: 0.7030000000000001 BRL


In [53]:
try:
    quotation(20, 'USD-BRL')
except Exception as e:
    # pass
    print(e)
else:
    print("request ok")

20 USD-BRL costs today: 105.40599999999999 BRL
request ok


In [64]:
currency_list = [
    "USD-BRL",
    "EUR-BRL",
    "BTC-BRL",
    "RPL-BRL", # inexistent
    "JPY-BRL",
]

for currency in currency_list:
    try:
        quotation(20, currency)
    except:
        #pass
        print(f"{currency} failed")

20 USD-BRL costs today: 105.40599999999999 BRL
20 EUR-BRL costs today: 102.834 BRL
20 BTC-BRL costs today: 2022.46 BRL
RPL-BRL failed
20 JPY-BRL costs today: 0.7030000000000001 BRL


In [67]:
def error_check(func): #a decorator
    def inner_func(*args, **kargs):
        try:
            func(*args, **kargs)
        except:
            print(f"{func.__name__} has failed.")
    return inner_func

In [68]:
@error_check
def quotation(value, currency):
    url = f"https://economia.awesomeapi.com.br/json/last/{currency}"
    ret = requests.get(url)
    currency_price = json.loads(ret.text)[currency.replace('-','')]
    print(f"{value} {currency:3} costs today: {float(currency_price['bid']) * 20} {currency[-3:]}")

In [69]:
currency_list = [
    "USD-BRL",
    "EUR-BRL",
    "BTC-BRL",
    "RPL-BRL", # inexistent
    "JPY-BRL",
]

for currency in currency_list:
    try:
        quotation(20, currency)
    except:
        #pass
        print(f"{currency} failed")

20 USD-BRL costs today: 105.40599999999999 BRL
20 EUR-BRL costs today: 102.834 BRL
20 BTC-BRL costs today: 2022.46 BRL
quotation has failed.
20 JPY-BRL costs today: 0.7030000000000001 BRL


In [12]:
# Example to explain args and kwargs in Python functions + using backoff package to handle error and retries
# max_time and max_tries
@backoff.on_exception(backoff.expo, (ConnectionAbortedError, ConnectionRefusedError, TimeoutError), max_tries=10)
def test_func(*args, **kargs):
    random_n = random.random()
    #print(f"""
    #        RND: {random_n}
    #        args: {args if args else 'no args'}
    #        kwargs: {kargs if kargs else 'no kwargs'}
    #""")
    log.debug(f"RND: {random_n}")
    log.info(f"args: {args if args else 'no args'}")
    log.info(f"kwargs: {kargs if kargs else 'no kwargs'}")
    
    if random_n < .2:
        log.error('Connection was finalized')
        raise ConnectionAbortedError('Connection was finalized')
    elif random_n < .4:
        log.error('Connection was denied')
        raise ConnectionRefusedError('Connection was denied')
    elif random_n < .6:
        log.error('Awaiting time exceed')
        raise TimeoutError('Awaiting time exceed')
    else:
        return "OK!"

In [17]:
test_func()

2022-10-20 09:01:47,613 - root - DEBUG - RND: 0.3296305709295171
2022-10-20 09:01:47,614 - root - INFO - args: no args
2022-10-20 09:01:47,615 - root - INFO - kwargs: no kwargs
2022-10-20 09:01:47,616 - root - ERROR - Connection was denied
2022-10-20 09:01:47,617 - backoff - INFO - Backing off test_func(...) for 0.1s (ConnectionRefusedError: Connection was denied)
2022-10-20 09:01:47,708 - root - DEBUG - RND: 0.4759640702100756
2022-10-20 09:01:47,710 - root - INFO - args: no args
2022-10-20 09:01:47,711 - root - INFO - kwargs: no kwargs
2022-10-20 09:01:47,712 - root - ERROR - Awaiting time exceed
2022-10-20 09:01:47,714 - backoff - INFO - Backing off test_func(...) for 0.2s (TimeoutError: Awaiting time exceed)
2022-10-20 09:01:47,959 - root - DEBUG - RND: 0.4478365097909812
2022-10-20 09:01:47,961 - root - INFO - args: no args
2022-10-20 09:01:47,962 - root - INFO - kwargs: no kwargs
2022-10-20 09:01:47,963 - root - ERROR - Awaiting time exceed
2022-10-20 09:01:47,964 - backoff - INF

'OK!'

In [14]:
test_func(52)

2022-10-20 09:01:28,333 - root - DEBUG - RND: 0.953872908746151
2022-10-20 09:01:28,334 - root - INFO - args: (52,)
2022-10-20 09:01:28,335 - root - INFO - kwargs: no kwargs


'OK!'

In [15]:
test_func(42, 51, name='will')

2022-10-20 09:01:31,689 - root - DEBUG - RND: 0.6771369931982685
2022-10-20 09:01:31,690 - root - INFO - args: (42, 51)
2022-10-20 09:01:31,691 - root - INFO - kwargs: {'name': 'will'}


'OK!'